**特征码41 0F BE 72 10找到 地址  大概是这种东西  13EED4A1054 - 41 0FBE 72 10         - movsx esi,byte ptr [r10+10]

In [209]:
import pymem
from pymem.pattern import pattern_scan_all


def aob_injection(process, aob):
    pm = pymem.Pymem(process)  # Open a handle to the process
    aob_address = pattern_scan_all(
        pm.process_handle, aob
    )  # Scan the entire memory of the process for the AOB
    print(aob_address)  # Print the address where the AOB was found
    print(hex(aob_address))

    return aob_address


#aob = bytes.fromhex("41 0F BE 71 10 45")  # Define the AOB r9+10
aob = bytes.fromhex("41 0F BE 72 10 45 0F")  # Define the AOB r10+10
aob_address=aob_injection(
    "javaw.exe", aob
)  # Call the function with the name of the process and the AOB
#打印出来的地址需要变成jmp

1996898335956
0x1d0f06a7cd4


开辟空间 叫他TR

In [210]:
pm = pymem.Pymem("javaw.exe")
TR = pymem.memory.allocate_memory(pm.process_handle, 4)
print(TR)
print(hex(TR))

1995211472896
0x1d08bdf0000


In [211]:
def format_address(address, length=8):
    reversed_bytes = address.to_bytes(length, byteorder='little')
    reversed_formatted = ' '.join(f'{b:02X}' for b in reversed_bytes)
    return reversed_formatted

#print(format_address(0x1CA99861000))
TR_address_reversed_formatted=format_address(TR)
print(TR_address_reversed_formatted)


00 00 DF 8B D0 01 00 00


In [212]:
push_rax=['50']
#lea_eax_rx_x=['41','8D','41','10']# r9+10
lea_eax_rx_x=['41','8D','42','10']# r10+10
move_TR_eax=['A3']+TR_address_reversed_formatted.split(' ')# [TR],EAX
pop_rax=['58']
#movsx_esi_byte_ptr_rx_x=['41','0F' ,'BE', '71', '10']
movsx_esi_byte_ptr_rx_x=['41','0F' ,'BE', '72', '10']


In [213]:
len('50 41 8D 41 10 A3 00 10 86 99 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC OB'.split(' '))
#'50 41 8D 41 10 A3 00 10 86 99 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC OB'.split(' ')

25

In [214]:
part_head_combine= push_rax+lea_eax_rx_x+move_TR_eax+pop_rax+movsx_esi_byte_ptr_rx_x
part_head_combine
print('len part_head_combine',len(part_head_combine))
print(part_head_combine)
jmp_place=['E9']

len part_head_combine 20
['50', '41', '8D', '42', '10', 'A3', '00', '00', 'DF', '8B', 'D0', '01', '00', '00', '58', '41', '0F', 'BE', '72', '10']


In [215]:
shell_code_len=len(part_head_combine)+1+4
print('shell_code_len',shell_code_len)

shell_code_len 25


In [216]:
# Allocate new memory
newmem = pymem.memory.allocate_memory(pm.process_handle, shell_code_len)

In [217]:
print(newmem)
print(hex(newmem))

1995211407360
0x1d08bde0000


In [218]:
newmen_end_row_address_jmp=hex(len(part_head_combine)+newmem)
newmen_end_row_address_jmp

'0x1d08bde0014'

In [219]:
import struct

def calculate_jmp_offset(jmp_addr, target_addr):
    offset = target_addr - (jmp_addr + 5)
    return offset

def to_signed_32_bit_le(i):
    return struct.pack('<i', i)

In [220]:
# 先算 newmen中jmp 的偏移量
jmp_addr_1=int(newmen_end_row_address_jmp,16)  #多数以14结尾
print('jmp_addr_1',jmp_addr_1)
target_addr_1=aob_address+5
print('target_addr_1',target_addr_1)
print(hex(target_addr_1))
print('offset_1',jmp_addr_1, target_addr_1)
offset_1 = calculate_jmp_offset(jmp_addr_1, target_addr_1)
print('offset_1',offset_1)
offset_1_le = to_signed_32_bit_le(offset_1)
print('offset_1_le',offset_1_le)

# offset_2_le 是从 aob_address 跳到 newmen 的偏移量 A开头
jmp_addr_2=aob_address  # aob address
print('jmp_addr_2',jmp_addr_2,hex(jmp_addr_2))
target_addr_2=newmem
print('target_addr_2',target_addr_2,hex(target_addr_2))
offset_2 = calculate_jmp_offset(jmp_addr_2, target_addr_2)
# Convert to signed 32-bit little endian format
offset_2_le = to_signed_32_bit_le(offset_2)
# Print the offset in hexadecimal
print(offset_2_le.hex())


jmp_addr_1 1995211407380
target_addr_1 1996898335961
0x1d0f06a7cd9
offset_1 1995211407380 1996898335961
offset_1 1686928576
offset_1_le b'\xc0|\x8cd'
jmp_addr_2 1996898335956 0x1d0f06a7cd4
target_addr_2 1995211407360 0x1d08bde0000
2783739b


In [221]:
hex_values=part_head_combine+jmp_place+[offset_1_le[0:1],offset_1_le[1:2],offset_1_le[2:3],offset_1_le[3:4]]
hex_values

['50',
 '41',
 '8D',
 '42',
 '10',
 'A3',
 '00',
 '00',
 'DF',
 '8B',
 'D0',
 '01',
 '00',
 '00',
 '58',
 '41',
 '0F',
 'BE',
 '72',
 '10',
 'E9',
 b'\xc0',
 b'|',
 b'\x8c',
 b'd']

In [222]:
#hex_string = "50 41 8D 41 10 A3 00 00 10 88 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC 0B"
#hex_values = hex_string.split()  # split the string into a list of two-character strings
hex_values=part_head_combine+jmp_place+[offset_1_le[0:1],offset_1_le[1:2],offset_1_le[2:3],offset_1_le[3:4]]
byte_values = []
for hv in hex_values:
    if isinstance(hv, str):  # if hv is a hex string
        byte_values.append(int(hv, 16))
    elif isinstance(hv, bytes):  # if hv is a byte object
        byte_values.append(int.from_bytes(hv, 'little'))  # or 'big' if your data is big-endian
shellcode = bytes(byte_values)
print(shellcode)


b'PA\x8dB\x10\xa3\x00\x00\xdf\x8b\xd0\x01\x00\x00XA\x0f\xber\x10\xe9\xc0|\x8cd'


先要创造shellcode 然后再写入

In [223]:
hex(newmem)

'0x1d08bde0000'

In [224]:
result1=pm.write_bytes(newmem, shellcode, len(shellcode))

In [225]:
#现在要去aob address jmp 到newmen
inject_hex = ['E9']+[offset_2_le[0:1],offset_2_le[1:2],offset_2_le[2:3],offset_2_le[3:4]]
print(inject_hex)
byte_values = []
for hv in inject_hex:
    if isinstance(hv, str):  # if hv is a hex string
        byte_values.append(int(hv, 16))
    elif isinstance(hv, bytes):  # if hv is a byte object
        byte_values.append(int.from_bytes(hv, 'little'))  # or 'big' if your data is big-endian
inject_hex_shellcode = bytes(byte_values)
print(inject_hex_shellcode)

['E9', b"'", b'\x83', b's', b'\x9b']
b"\xe9'\x83s\x9b"


In [226]:
inject_hex

['E9', b"'", b'\x83', b's', b'\x9b']

In [227]:
print('aob_address',aob_address,hex(aob_address))

aob_address 1996898335956 0x1d0f06a7cd4


In [228]:
len(inject_hex_shellcode)
print(inject_hex_shellcode)

b"\xe9'\x83s\x9b"


In [229]:
inject_hex_shellcode

b"\xe9'\x83s\x9b"

In [230]:
print(aob_address, inject_hex_shellcode, len(inject_hex_shellcode))

1996898335956 b"\xe9'\x83s\x9b" 5


In [231]:
result2=pm.write_bytes(aob_address, inject_hex_shellcode, len(inject_hex_shellcode))
print(result2)

None


In [232]:
hex(TR)

'0x1d08bdf0000'

In [234]:
import pymem

# Initialize pymem
pm = pymem.Pymem('javaw.exe')  # Replace 'YourProcessName.exe' with the name of your process

# Define the memory address
address = TR

# Read bytes from the memory
# Assuming you want to read 4 bytes (an integer) at that address
data = pm.read_bytes(address, 4)
print(data)

# Convert the bytes to an integer
value = int.from_bytes(data, byteorder='little')  # Use 'big' if the byteorder is big endian

print(value)


b'\xf0\x90\xce\xea'
3939406064


In [280]:
data = pm.read_bytes(3939406064-4, 10)

In [272]:
data

b'\x06\x00\r\x00\x012\x0b\x00\x02\x03'

In [267]:
# Convert the first 2 bytes to an integer
value = int.from_bytes(data[:2], byteorder='little')  # Use 'big' if the byteorder is big endian

print(value)  # Output: 12081

6


In [268]:
# Convert the first 2 bytes to an integer
value = int.from_bytes(data[2:4], byteorder='little')  # Use 'big' if the byteorder is big endian

print(value)  # Output: 12081

13


In [269]:
value = int.from_bytes(data[4:6], byteorder='little')  # Use 'big' if the byteorder is big endian
print(value)  # Output: 12081

12801


In [273]:
value = int.from_bytes(data[8:], byteorder='little')  # Use 'big' if the byteorder is big endian
print(value)  # Output: 770

770


In [281]:
hex_value = hex(data[9])
hex_value[-3] #3 right,0 down

'0x3'